In [1]:
import sys
import torch

if "google.colab" in sys.modules:
    from google.colab import drive
    drive.mount("/content/drive")
    %cd "/content/drive/MyDrive/Python/Bath University/RL1_CW/Louie/TD3"
    !pip install swig
    !pip install gymnasium[box2d]

from Agent import TD3Agent
from Networks import Critic, Actor

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Mounted at /content/drive
/content/drive/MyDrive/Python/Bath University/RL1_CW/Louie/TD3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 24.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp312-cp312-linux_x86_64.whl size=2399055 sha256=666aa2414e45fa2173abf7c87040b1b69d6744259c323ce22607c6f3f2529b34
  Stored in directory: /root/.cache/pip/wheels/2a/e9/60/774da0bcd07f7dc7761a8590fa2d065e4069568e78dcdc3318
Successfully built box2d-py


# Training - Stage 1 - Complete Normal Mode

In [7]:
actor = Actor()
critic1 = Critic()
critic2 = Critic()

agent = TD3Agent(critic_network1=critic1,
                 critic_network2=critic2,
                 actor_network=actor,
                 device=device,
                 hardcore=False,
                 max_buffer_length=1_000_000)

agent.learn(n_episodes=2000,
            discount_factor=0.99,
            minibatch_size=256,
            tau=0.005,
            random_exploration_steps=10_000,
            actor_exploration_steps=0,
            vid_every=50,
            stop_after=1,
            reset_optim=True,
            reset_buffer=True,
            # ====================== #
            critic_lr=1e-3,
            actor_lr=1e-4,
            critic_grad_clip=1.0,
            actor_grad_clip=1.0,
            exploratory_noise=0.1,
            exploratory_noise_clip=0.3,
            policy_noise=0.2,
            policy_noise_clip=0.5,
            policy_delay=2,
            updates_per_step=1)

Performing Random Exploration...
Performing Actor Exploration...
Running Episode 1...
Reward: -119.64 - Step Count: 408 - Run Time: 3.41s - Total Step Count - 408

========TEST RUN========


/usr/local/lib/python3.12/dist-packages/gymnasium/wrappers/rendering.py:293: UserWarning: WARN: Overwriting existing videos at /content/drive/MyDrive/Python/Bath University/RL1_CW/Louie/TD3/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Reward: -110.99 - Step Count: 139 - Run Time: 0.99s

Running Episode 2...
Reward: -122.99 - Step Count: 142 - Run Time: 0.97s - Total Step Count - 550
Running Episode 3...
Reward: -112.06 - Step Count: 132 - Run Time: 0.90s - Total Step Count - 682
Running Episode 4...
Reward: -122.06 - Step Count: 78 - Run Time: 0.53s - Total Step Count - 760
Running Episode 5...
Reward: -126.60 - Step Count: 98 - Run Time: 0.67s - Total Step Count - 858
Running Episode 6...
Reward: -123.43 - Step Count: 100 - Run Time: 0.69s - Total Step Count - 958
Running Episode 7...
Reward: -123.58 - Step Count: 93 - Run Time: 0.64s - Total Step Count - 1051
Running Episode 8...
Reward: -113.02 - Step Count: 64 - Run Time: 0.44s - Total Step Count - 1115
Running Episode 9...
Reward: -118.50 - Step Count: 52 - Run Time: 0.35s - Total Step Count - 1167
Running Episode 10...
Reward: -120.61 - Step Count: 43 - Run Time: 0.29s - Total Step Count - 1210
Running Episode 11...
Reward: -137.88 - Step Count: 160 - Run Time

# Training - Stage 2 - Go Faster

In [8]:
agent.learn(n_episodes=500,
            discount_factor=0.99,
            minibatch_size=256,
            tau=0.005,
            random_exploration_steps=0,
            actor_exploration_steps=0,
            vid_every=50,
            stop_after=None,
            reset_optim=False,
            reset_buffer=False,
            # ====================== #
            critic_lr=1e-3,
            actor_lr=1e-4,
            critic_grad_clip=1.0,
            actor_grad_clip=1.0,
            exploratory_noise=0.1,
            exploratory_noise_clip=0.3,
            policy_noise=0.2,
            policy_noise_clip=0.5,
            policy_delay=2,
            updates_per_step=1)

Performing Random Exploration...
Performing Actor Exploration...
Running Episode 1...
Reward: 297.57 - Step Count: 932 - Run Time: 7.01s - Total Step Count - 932

========TEST RUN========
Reward: 302.56 - Step Count: 849 - Run Time: 10.61s

Running Episode 2...
Reward: 297.50 - Step Count: 941 - Run Time: 7.09s - Total Step Count - 1873
Running Episode 3...
Reward: 296.09 - Step Count: 953 - Run Time: 6.44s - Total Step Count - 2826
Running Episode 4...
Reward: 297.10 - Step Count: 970 - Run Time: 7.27s - Total Step Count - 3796
Running Episode 5...
Reward: 297.85 - Step Count: 934 - Run Time: 6.35s - Total Step Count - 4730
Running Episode 6...
Reward: 295.06 - Step Count: 965 - Run Time: 7.28s - Total Step Count - 5695
Running Episode 7...
Reward: 296.05 - Step Count: 990 - Run Time: 6.75s - Total Step Count - 6685
Running Episode 8...
Reward: 299.77 - Step Count: 916 - Run Time: 6.84s - Total Step Count - 7601
Running Episode 9...
Reward: 298.62 - Step Count: 944 - Run Time: 6.37s -

# Training - Stage 3 - Complete Hardcore Mode



In [10]:
critic1 = Critic()
critic2 = Critic()

stage2_actor = Actor()
stage2_actor.load_state_dict(torch.load("outputs/stage 2/actor_network.pth", map_location=device))

agent = TD3Agent(critic_network1=critic1,
                 critic_network2=critic2,
                 actor_network=stage2_actor,
                 device=device,
                 hardcore=True,
                 max_buffer_length=1_000_000)

agent.learn(n_episodes=2000,
            discount_factor=0.99,
            minibatch_size=256,
            tau=0.005,
            random_exploration_steps=10_000,
            actor_exploration_steps=0,
            vid_every=50,
            stop_after=1,
            reset_optim=True,
            reset_buffer=True,
            # ====================== #
            critic_lr=1e-3,
            actor_lr=1e-4,
            critic_grad_clip=1.0,
            actor_grad_clip=1.0,
            exploratory_noise=0.1,
            exploratory_noise_clip=0.3,
            policy_noise=0.2,
            policy_noise_clip=0.5,
            policy_delay=2,
            updates_per_step=1)

Performing Random Exploration...
Performing Actor Exploration...
Running Episode 1...
Reward: -92.66 - Step Count: 119 - Run Time: 1.01s - Total Step Count - 119

========TEST RUN========
Reward: -139.82 - Step Count: 176 - Run Time: 2.56s

Running Episode 2...
Reward: -129.33 - Step Count: 125 - Run Time: 0.85s - Total Step Count - 244
Running Episode 3...
Reward: -115.46 - Step Count: 63 - Run Time: 0.43s - Total Step Count - 307
Running Episode 4...
Reward: -120.13 - Step Count: 152 - Run Time: 1.02s - Total Step Count - 459
Running Episode 5...
Reward: -113.50 - Step Count: 98 - Run Time: 0.69s - Total Step Count - 557
Running Episode 6...
Reward: -113.92 - Step Count: 105 - Run Time: 0.77s - Total Step Count - 662
Running Episode 7...
Reward: -113.79 - Step Count: 108 - Run Time: 0.77s - Total Step Count - 770
Running Episode 8...
Reward: -115.93 - Step Count: 120 - Run Time: 0.89s - Total Step Count - 890
Running Episode 9...
Reward: -147.75 - Step Count: 1600 - Run Time: 11.75s 